# Understanding Bayes Nets

We have:

- Nodes: Variables
- Edges: Dependencies between variables

Bayes Nets encode the joint probability distribution of variables, allowing inference given evidence.

# Setup

In [96]:
from inspect import getsource
from IPython.display import display
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import itertools
import math
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
import time
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import json

In [97]:
random.seed(42)

In [98]:
df = pd.read_csv('../data/covid_preprocessed.csv')

In [99]:
df.columns

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'DATE_DIED', 'INTUBED',
       'PNEUMONIA', 'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU', 'AGE_GROUP'],
      dtype='object')

In [100]:
train_data, test_data = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["CLASIFFICATION_FINAL"]
)

In [101]:
categorical_columns = ['AGE_GROUP', 'PNEUMONIA', 'ICU', 'CLASIFFICATION_FINAL', 'SEX', 'OBESITY', 'DIABETES']
for col in categorical_columns:
    train_data[col] = train_data[col].astype("category")
    test_data[col] = test_data[col].astype("category")

In [102]:
for col in categorical_columns:
    train_data[col] = train_data[col].cat.remove_unused_categories()
    test_data[col] = test_data[col].cat.remove_unused_categories()

# Helper Code

In [103]:
def extend(s, var, val):
    """Create a copy of dictionary `s` and add a new key-value pair where `var` is set to `val`. Return the updated copy."""
    return {**s, var: val}

In [106]:
class ProbDist:
    """
    Represents a discrete probability distribution for a single random variable. 
    You can initialize it with a variable name and an optional frequency dictionary.
    Probabilities are normalized automatically if frequencies are provided.

    Example:
    >>> P = ProbDist('Flip'); P['H'], P['T'] = 0.25, 0.75; P['H']
    0.25
    >>> P = ProbDist('X', {'lo': 125, 'med': 375, 'hi': 500})
    >>> P['lo'], P['med'], P['hi']
    (0.125, 0.375, 0.5)
    """
    def __init__(self, varname='?', freqs=None):
        """
        Initialize the distribution. If `freqs` is given, it must be a dictionary 
        with values as keys and their frequencies as values. The distribution is normalized.
        """
        self.prob = {}
        self.varname = varname
        self.values = []
        if freqs:
            for (v, p) in freqs.items():
                self[v] = p
            self.normalize()

    def __getitem__(self, val):
        """Retrieve the probability of `val` if it exists, otherwise return 0."""
        try:
            return self.prob[val]
        except KeyError:
            return 0

    def __setitem__(self, val, p):
        """Assign probability `p` to the value `val`."""
        if val not in self.values:
            self.values.append(val)
        self.prob[val] = p

    def normalize(self):
        """
        Ensure that the probabilities of all values sum up to 1. 
        If the sum of values is 0, a ZeroDivisionError is raised.
        """
        total = sum(self.prob.values())
        if not np.isclose(total, 1.0):
            for val in self.prob:
                self.prob[val] /= total
        return self

    def show_approx(self, numfmt='{:.3g}'):
        """
        Display the probabilities rounded to a specified format, sorted by their keys. 
        Useful for readability in doctests.
        """
        return ', '.join([('{}: ' + numfmt).format(v, p)
                          for (v, p) in sorted(self.prob.items())])

    def __repr__(self):
        """Return a string representation of the distribution."""
        return "P({})".format(self.varname)

In [109]:
def probability_sampling(probabilities):
    """
    Perform random sampling based on the given probability distribution. 
    Returns an outcome based on the probabilities.
    """
    total = sum(probabilities.values())
    r = random.uniform(0, total)
    cumulative = 0
    for outcome, prob in probabilities.items():
        cumulative += prob
        if r <= cumulative:
            return outcome
    return None  # This should not occur if probabilities are normalized.

In [110]:
class MultiClassBayesNode:
    """
    Represents a node in a Bayesian network for multi-class variables. 
    It contains the variable, its parents, and the conditional probability table (CPT).
    """
    def __init__(self, X, parents, cpt):
        """
        Initialize the node with:
        - `X`: Variable name.
        - `parents`: List of parent variable names.
        - `cpt`: A dictionary representing the conditional probability table.
        """
        if isinstance(parents, str):
            parents = parents.split()
        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """
        Compute the conditional probability of `X=value` given the parent values in `event`.
        """
        parent_values = tuple(event.get(p, None) for p in self.parents)
        probabilities = self.cpt.get(parent_values, {})
        return probabilities.get(value, 0)  # Defaults to 0 if `value` is not found.

    def sample(self, event):
        """
        Sample a value for the variable given parent values in `event`. 
        Sampling is based on the conditional probability distribution.
        """
        parent_values = tuple(event.get(p, None) for p in self.parents)
        probabilities = self.cpt.get(parent_values, {})
        return probability_sampling(probabilities)

    def __repr__(self):
        """Return a string representation of the node."""
        return repr((self.variable, ' '.join(self.parents)))

In [111]:
class BayesNet:
    """
    Represents a Bayesian network consisting of nodes (variables) and their dependencies.
    Supports multi-class nodes.
    """
    def __init__(self, node_specs=None):
        """
        Initialize the network. Nodes must be added in topological order 
        (parents must be added before their children).
        """
        self.nodes = []
        self.variables = []
        node_specs = node_specs or []
        for node_spec in node_specs:
            self.add(node_spec)

    def add(self, node_spec):
        """
        Add a node to the network. Accepts either a pre-constructed node 
        or the specifications for a new node.
        """
        if isinstance(node_spec, MultiClassBayesNode):
            node = node_spec
        else:
            node = MultiClassBayesNode(*node_spec)

        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)

        self.nodes.append(node)
        self.variables.append(node.variable)

        # Register this node as a child for its parent nodes
        for parent in node.parents:
            self.variable_node(parent).children.append(node)

    def variable_node(self, var):
        """Retrieve the node corresponding to the variable `var`."""
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception(f"No such variable: {var}")

    def variable_values(self, var):
        """Retrieve the domain of `var` (default is `[True, False]`)."""
        return [True, False]

    def __repr__(self):
        """Return a string representation of the network."""
        return f"BayesNet({self.nodes!r})"

In [113]:
class Factor:
    """Represents a factor in a joint distribution."""
    def __init__(self, variables, cpt):
        """
        Initialize the factor with:
        - `variables`: List of variables involved in the factor.
        - `cpt`: Conditional probability table.
        """
        self.variables = variables
        self.cpt = cpt

    def normalize(self):
        """
        Normalize the factor and return a `ProbDist` for the remaining variable.
        This is only valid if the factor has one variable left.
        """
        assert len(self.variables) == 1
        return ProbDist(self.variables[0], {k: v for ((k,), v) in self.cpt.items()})

    def p(self, e):
        """Retrieve the probability for the event `e` from the factor's CPT."""
        return self.cpt[event_values(e, self.variables)]

In [114]:
def enumerate_all(variables, e, bn):
    """
    Calculate the sum of all entries in the joint probability distribution 
    for `variables` consistent with the evidence `e` in network `bn`.
    """
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.variable_node(Y)
    if Y in e:
        return Ynode.p(e[Y], e) * enumerate_all(rest, e, bn)
    else:
        return sum(Ynode.p(y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.variable_values(Y))

In [115]:
def enumeration_ask(X, e, bn):
    """
    Compute the conditional probability distribution for the query variable `X` 
    given evidence `e` in the Bayesian network `bn`.
    """
    assert X not in e, "Query variable must not overlap with the evidence."
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()

In [123]:
def event_values(event, variables):
    """
    Generate a tuple containing the values of the specified variables from the event.
    
    Examples:
    >>> event_values({'A': 10, 'B': 9, 'C': 8}, ['C', 'A'])
    (8, 10)
    >>> event_values((1, 2), ['C', 'A'])
    (1, 2)
    """
    if isinstance(event, tuple) and len(event) == len(variables):
        return event
    else:
        return tuple(event[var] for var in variables)

# Design the Network Structure

Find dependencies.

# Estimate Conditional Probabilities

If a node is a root node, then estimate probability directly from the data. Estimate conditional probabilities based on parent for non-root nodes.

In [124]:
def compute_cpt(data, target, parents, alpha=1):
    """
    Compute CPT with Laplace smoothing.
    
    Args:
        
        data: pandas DataFrame (training data)
        target: str, target variable
        parents: list of parent variable names
        alpha: smoothing parameter (default=1)
    
    Returns:
        cpt: dict { parent_values_tuple: { target_value: probability } }
    """
    target_values = data[target].cat.categories
    
    print("TARGET VALUES: ", target_values)

    if not parents:
        # Marginal distribution of target
        counts = defaultdict(lambda: alpha)
        for val in data[target]:
            counts[val] += 1
        total = sum(counts.values())
        cpt = {(): {tv: counts[tv]/total for tv in counts}}
        return cpt

    # Determine possible parent combinations
    from itertools import product
    parent_values_list = [data[p].cat.categories for p in parents]
    print("PARENT VALUES LIST", parent_values_list)
    print("PARENTS", parents)
    print("data DIABATES CATEOGORIES", data['DIABETES'].cat.categories)
    parent_combinations = list(product(*parent_values_list)) if parents else [()]

    # Initialize counts with alpha
    counts = {pc: defaultdict(lambda: alpha) for pc in parent_combinations}

    # Count occurrences
    for _, row in data.iterrows():
        pv = tuple(row[p] for p in parents) if parents else ()
        tv = row[target]
        counts[pv][tv] += 1

    # Compute probabilities
    cpt = {}
    for pc in parent_combinations:
        total = sum(counts[pc].values())
        cpt[pc] = {tv: (counts[pc][tv] / total) for tv in counts[pc]}
        
    return cpt

In [125]:
def compute_all_cpts(train_data):
    """
    Compute all CPTs and record the time taken.
    """
    start_time = time.time()  # Start timing
    
    # Compute CPTs
    cpt_icu = compute_cpt(train_data, 'ICU', ['PNEUMONIA'])
    cpt_pneumonia = compute_cpt(train_data, 'PNEUMONIA', ['AGE_GROUP'])
    cpt_age_group = compute_cpt(train_data, 'AGE_GROUP', [])
    cpt_sex_group = compute_cpt(train_data, 'SEX', [])
    cpt_obesity_group = compute_cpt(train_data, 'OBESITY', [])
    cpt_diabetes_group = compute_cpt(train_data, 'DIABETES', [])
    cpt_classification = compute_cpt(train_data, 'CLASIFFICATION_FINAL', ['SEX', 'OBESITY', 'DIABETES', 'ICU'])
    
    end_time = time.time()  # End timing
    training_time = end_time - start_time
    print(f"Training Time (CPT Computation): {training_time:.4f} seconds")

    return {
        "cpt_classification": cpt_classification,
        "cpt_icu": cpt_icu,
        "cpt_pneumonia": cpt_pneumonia,
        "cpt_age_group": cpt_age_group,
        "cpt_sex_group": cpt_sex_group,
        "cpt_obesity_group": cpt_obesity_group,
        "cpt_diabetes_group": cpt_diabetes_group
    }, training_time

In [126]:
# Measure Training Time
cpts, training_time = compute_all_cpts(train_data)

TARGET VALUES:  Index([1, 2], dtype='int64')
PARENT VALUES LIST [Index([1, 2], dtype='int64')]
PARENTS ['PNEUMONIA']
data DIABATES CATEOGORIES Index([1, 2], dtype='int64')
TARGET VALUES:  Index([1, 2], dtype='int64')
PARENT VALUES LIST [Index([0, 1, 2, 3, 4], dtype='int64')]
PARENTS ['AGE_GROUP']
data DIABATES CATEOGORIES Index([1, 2], dtype='int64')
TARGET VALUES:  Index([0, 1, 2, 3, 4], dtype='int64')
TARGET VALUES:  Index([1, 2], dtype='int64')
TARGET VALUES:  Index([1, 2], dtype='int64')
TARGET VALUES:  Index([1, 2], dtype='int64')
TARGET VALUES:  Index([0, 1], dtype='int64')
PARENT VALUES LIST [Index([1, 2], dtype='int64'), Index([1, 2], dtype='int64'), Index([1, 2], dtype='int64'), Index([1, 2], dtype='int64')]
PARENTS ['SEX', 'OBESITY', 'DIABETES', 'ICU']
data DIABATES CATEOGORIES Index([1, 2], dtype='int64')
Training Time (CPT Computation): 109.4886 seconds


# Implement Inference

Use the chain rule of probability to predict target given evidence.

In [127]:
classification_node = MultiClassBayesNode("CLASIFFICATION_FINAL", ['SEX', 'OBESITY', 'DIABETES', 'ICU'], cpts["cpt_classification"])
pneumonia_node = MultiClassBayesNode("PNEUMONIA", ["AGE_GROUP"], cpts["cpt_pneumonia"])
icu_node = MultiClassBayesNode("ICU", ["PNEUMONIA"], cpts["cpt_icu"])
age_node = MultiClassBayesNode("AGE_GROUP", [], cpts["cpt_age_group"])
obesity_node = MultiClassBayesNode("OBESITY", [], cpts["cpt_obesity_group"])
sex_node = MultiClassBayesNode("SEX", [], cpts["cpt_sex_group"])
diabetes_node = MultiClassBayesNode("DIABETES", [], cpts["cpt_diabetes_group"])

In [128]:
classification_final = BayesNet([
    age_node,
    sex_node,
    obesity_node,
    diabetes_node,
    pneumonia_node,
    icu_node,
    classification_node
])

# Evaluate the Model

In [129]:
def predict_bayes_net(bn, evidence, query_var):
    """
    Predict the most likely value of a query variable given evidence using the Bayesian Network.
    
    Args:
        bn: Bayesian network.
        evidence: Dictionary of evidence variables and their values.
        query_var: Variable to predict.
    
    Returns:
        The most likely value of the query variable.
    """
    result = enumeration_ask(query_var, evidence, bn)
    return max(result.prob, key=lambda k: result.prob[k])

In [130]:
def evaluate_bayes_net(bn, test_data, query_var):
    """
    Evaluate the Bayesian Network on a test dataset.
    
    Args:
        bn: Bayesian Network (BayesNet instance).
        test_data: Test dataset (pandas DataFrame).
        query_var: The target variable to predict.
    
    Returns:
        Accuracy of the predictions as a float.
    """
    correct = 0  # Count of correct predictions

    for _, row in test_data.iterrows():
        # Build evidence dictionary from test row
        evidence = {
            "AGE_GROUP": row["AGE_GROUP"],
            "SEX": row["SEX"],
            "OBESITY": row["OBESITY"],
            "DIABETES": row["DIABETES"],
            "PNEUMONIA": row["PNEUMONIA"],
            "ICU": row["ICU"]
        }

        # Predict the target variable
        prediction = predict_bayes_net(bn, evidence, query_var)
        
        
        # Check if the prediction matches the actual value
        if prediction == row[query_var]:
            correct += 1

    # Calculate accuracy
    accuracy = correct / len(test_data)
    return accuracy

In [131]:
def evaluate_bayes_net_with_time(bn, test_data, query_var):
    """
    Evaluate the Bayesian Network on a test dataset and measure prediction time.
    """
    y_true = []
    y_pred = []

    # Start Timing Predictions
    start_time = time.time()

    for _, row in test_data.iterrows():
        evidence = {
            "AGE_GROUP": row["AGE_GROUP"],
            "SEX": row["SEX"],
            "OBESITY": row["OBESITY"],
            "DIABETES": row["DIABETES"],
            "PNEUMONIA": row["PNEUMONIA"],
            "ICU": row["ICU"]
        }
        prediction = predict_bayes_net(bn, evidence, query_var)
        y_true.append(row[query_var])
        y_pred.append(prediction)

    end_time = time.time()
    prediction_time = end_time - start_time

    # Calculate Metrics
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, zero_division=0)

    print("Confusion Matrix:\n", cm)
    print("\nClassification Report:\n", report)
    print(f"Prediction Time: {prediction_time:.4f} seconds")

    metrics = {
        "accuracy": acc,
        "prediction_time": prediction_time
    }
    return metrics

In [132]:
# Measure Prediction Time
metrics = evaluate_bayes_net_with_time(classification_final, test_data, "CLASIFFICATION_FINAL")

# Final Output
print("\n--- Final Results ---")
print(f"Training Time: {training_time:.4f} seconds")
print(f"Prediction Time: {metrics['prediction_time']:.4f} seconds")
print(f"Accuracy: {metrics['accuracy']:.2%}")

Confusion Matrix:
 [[124074   7245]
 [ 69890   8506]]

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.94      0.76    131319
           1       0.54      0.11      0.18     78396

    accuracy                           0.63    209715
   macro avg       0.59      0.53      0.47    209715
weighted avg       0.60      0.63      0.55    209715

Prediction Time: 25.3956 seconds

--- Final Results ---
Training Time: 109.4886 seconds
Prediction Time: 25.3956 seconds
Accuracy: 63.22%


# For Later

In [133]:
cpts

{'cpt_classification': {(1, 1, 1, 1): {1: 0.6524822695035462,
   0: 0.3475177304964539},
  (1, 1, 1, 2): {0: 0.5135019593872462, 1: 0.4864980406127538},
  (1, 1, 2, 1): {0: 0.32945736434108525, 1: 0.6705426356589147},
  (1, 1, 2, 2): {0: 0.5988674268869754, 1: 0.4011325731130246},
  (1, 2, 1, 1): {1: 0.6389157792836399, 0: 0.3610842207163601},
  (1, 2, 1, 2): {0: 0.546855527523633, 1: 0.4531444724763671},
  (1, 2, 2, 1): {1: 0.5164705882352941, 0: 0.4835294117647059},
  (1, 2, 2, 2): {1: 0.3181739188134517, 0: 0.6818260811865483},
  (2, 1, 1, 1): {0: 0.3108108108108108, 1: 0.6891891891891891},
  (2, 1, 1, 2): {0: 0.45563591462339037, 1: 0.5443640853766096},
  (2, 1, 2, 1): {1: 0.7348314606741573, 0: 0.2651685393258427},
  (2, 1, 2, 2): {0: 0.53274414123856, 1: 0.46725585876144005},
  (2, 2, 1, 1): {0: 0.32887402452619846, 1: 0.6711259754738016},
  (2, 2, 1, 2): {0: 0.49224692412394244, 1: 0.5077530758760576},
  (2, 2, 2, 1): {0: 0.4107517849643007, 1: 0.5892482150356992},
  (2, 2, 2, 2

In [134]:
def cpts_to_json(cpts):
    serializable_cpts = {}
    for var, cpt in cpts.items():
        serializable_cpts[var] = {
            str(parent_comb): {str(k): v for k, v in target_probs.items()}
            for parent_comb, target_probs in cpt.items()
        }
    return serializable_cpts

In [137]:
# Save the CPTs to a JSON file
cpt_json = cpts_to_json(cpts)
with open("covid_cpts.json", "w") as f:
    json.dump(cpt_json, f, indent=4)
print("CPTs saved successfully to covid_cpts.json!")

CPTs saved successfully to covid_cpts.json!


In [138]:
cpt_json

{'cpt_classification': {'(1, 1, 1, 1)': {'1': 0.6524822695035462,
   '0': 0.3475177304964539},
  '(1, 1, 1, 2)': {'0': 0.5135019593872462, '1': 0.4864980406127538},
  '(1, 1, 2, 1)': {'0': 0.32945736434108525, '1': 0.6705426356589147},
  '(1, 1, 2, 2)': {'0': 0.5988674268869754, '1': 0.4011325731130246},
  '(1, 2, 1, 1)': {'1': 0.6389157792836399, '0': 0.3610842207163601},
  '(1, 2, 1, 2)': {'0': 0.546855527523633, '1': 0.4531444724763671},
  '(1, 2, 2, 1)': {'1': 0.5164705882352941, '0': 0.4835294117647059},
  '(1, 2, 2, 2)': {'1': 0.3181739188134517, '0': 0.6818260811865483},
  '(2, 1, 1, 1)': {'0': 0.3108108108108108, '1': 0.6891891891891891},
  '(2, 1, 1, 2)': {'0': 0.45563591462339037, '1': 0.5443640853766096},
  '(2, 1, 2, 1)': {'1': 0.7348314606741573, '0': 0.2651685393258427},
  '(2, 1, 2, 2)': {'0': 0.53274414123856, '1': 0.46725585876144005},
  '(2, 2, 1, 1)': {'0': 0.32887402452619846, '1': 0.6711259754738016},
  '(2, 2, 1, 2)': {'0': 0.49224692412394244, '1': 0.507753075876